# Fonctions utiles

In [2]:
#Pour une déliberation et les arrêtés
import os
import re
import fitz

# TODO vérifier si Tuple est important
#from typing import Tuple

# Librairies pour transformer les tableaux
import camelot
from bs4 import BeautifulSoup

# Librairies pour récupération des délibérations déjà publiées
import requests
import json

# Librairie pour export des résultats en fichier CSV
from csv import DictWriter

# Librairie pour obtenir la date du jour (date de publication)
from datetime import datetime

# Librairie pour décoder les PDF encodés
from pikepdf import Pdf

# Librairie pour détection anciens contrôles légalité image
from matplotlib import pyplot as plt
import platform
import cv2
from PIL import Image
from tempfile import TemporaryDirectory, NamedTemporaryFile
import pytesseract
from pathlib import Path
if platform.system() == "Windows":
    #pytesseract.pytesseract.tesseract_cmd = (r"C:/Program Files/Tesseract-OCR/tesseract.exe")
    # Windows also needs poppler_exe
    path_to_poppler_exe = Path(r'C:\Users\ancarette\Python Scripts\poppler-22.04.0\Library\bin')
    # Put our output files in a sane place...
    out_directory = Path(r"~\Desktop").expanduser()
else:
    out_directory = Path("~").expanduser() 

In [3]:
# Librairies inutiles utilisées antérieurement
# from pathlib import Path
# from tempfile import TemporaryDirectory, NamedTemporaryFile
# import pandas as pd
# from pdf2docx import parse


In [4]:

# @TODO: passer en argument l'utilisation de camelot


In [5]:
liste_actions_budgetaires = ['10400304','10401504','10401001','15200102','352001083','15000202','15000204','15200109','352001043','376002028','476002036','10200504','10200510','10200512','15100310','2510030719','10200301','10200304','13100206','13100401','2310040215','331004024','13100505','13100904','13101301','13101302','13101303','13101601','19100107','394001031','15600102','19100102','19100103','19100206','2910020409','391002034','391005011','19100701','19200207','19200902','19400103','19400114','14200101','14200302','14200308','2420040117','2420040217','313003011','14000402','14100103','14100106','14100112','14100401','37100903S','475001053','17600102','2760010307','17600501','17700104','17800201','19300402','19400301','17100203','17100301','17200101','17200104','17200108','17600401','17600601','2780020307','2930010113','12200803','12202903','12202904','2280050312','12201708','12202002','12203106','12800401','12800510','12800601','2230020111','11100405','11100302','19101001','11000205','2100020112','2100020212','2110050112','11100604','2110060112','2110060516','11100601','11100801','11100606','391010011','15400201','15400210','15400401','2540040214','15400501','15400601','2770010105','18101502','481015011','18200201','2820020104','18200501','13101001','13101002','10202501','10200406','10201901','10202303','10201401','10200801','10200403','10202102','12202201','12202404','15100202','2510030119','2510030219','15100401','392001024','19200503','19200701','17600206','376002058','13700102','2370010218','2570010218','15700406','10200203','11100610','13200201','13200205','13200306','13300201','18100101','18100701','381010015','18101101','48101302S','18102002','18101801','388003015','388004015','19500101','19500102','19500103','19500108','16200601','2930011113','19300308','10401301','10401902','15000101','15000201','15200110','452001086','452001096','15200204','40401801S','15000205','15000601','15200111','2520030121','10200508','13100101','13100202','13100204','2310020215','2310070115','13101503','2310040315','13100503','13101101','13101201','13101005','15600101','356001024','19100201','19101302','19400101','19400110','2940010109','391001024','19100301','19100605','19200206','14100205','14100301','342002032','14200310','11101101','11300102','11300303','14100109','14200304','472001053','17200201','2740020207','375001067','375001077','17600105','476003023','477003011','19300102','19300403','17100501','17100805','17200102','17400504','474007013','17600104','2760030607','376003038','476006013','393003036','12200401','12201101','12201201','12201301','12202901','12203202','12203206','12203401','328005031','12203103','12203104','12800406','12800506','2280050112','12300301','12300901','12300902','11100401','11100403','2110040112','11100701','11100501','11100503','11200401','2540030114','15400402','15400403','15400407','2540050214','15400602','17700101','477001011','2780010107','478001011','481015021','18200103','482001031','482006011','80223A','10201104','10201203','10201204','10201206','10200904','10201001','15100301','392007014','19200104','19200109','19200602','15300110','17600205','2570020218','13200101','13200103','13200601','13200602','18100201','481006011','2810100101','18101201','48800101S','488006011','16200301','16200802','16100302','16300101','19300112','19300114','19300116','493003076','10400307','10400801','10401701','352001093','15200203','15200103','2520010720','452001093','10200302','331001014','13100203','13100404','13100901','13100905','456001024','19100101','19100106','391002014','392002024','19200502','19400109','19400112','15600301','19100203','2910030109','391006021','2920020311','392002034','492005014','19400105','19400111','11300104','14200201','14200309','342004022','14000204','14000404','14000501','14100107','14200601','17100703','37400101S','17400401','17400601','2760010107','17600306','477002021','19300501','17100803','17100804','17300102','2740050207','17400701','17800202','2780020207','19300105','19300201','12800302','12201806','12202101','12203101','12203108','423002012','323003014','12300601','12300903','11100101','19101002','11000208','11100607','2110070112','11100609','11100611','11100901','11200301','19101203','19101204','15400203','15400206','2520010604','477002011','18101401','18102301','18200101','10201101','10201207','10201601','12202303','15100303','15100402','19200101','392001054','19200205','392007024','19200801','19200108','2920060111','15300202','12700101','13700101','10201214','13200104','13300105','13200305','13300202','18100401','2810070102','2810110102','18000102','18000103','18102201','18800302','19500107','19500110','2950010110','16100202','16100301','16100601','2930010813','19300406','19300305','10400308','10401101','10401302','10401303','10401901','2040030508','15200107','15200112','352001073','15300101','376002018','15000104','2520010320','2760020207','10200506','10200306','10200505','10200511','14000102','15100308','15100604','10200303','13100806','13101004','19400102','19400107','391002051','19100501','19100602','19100606','19200204','392005014','19400202','11300109','11300205','14100101','2410010117','14200102','14200202','14200404','14100113','14100501','17100102','371009058','471009023','17200109','2740010107','47400209S','37400401S','375001017','17600301','17600302','477001051','17200107','17200110','17400501','375001037','17600303','393001036','19300303','2930030113','12201501','2220040100','12200602','12200701','12200802','12203002','12203003','12203004','12203403','12201701','12201704','12201709','12201804','12201601','12203301','12800301','12800501','12800507','12800508','12800701','423001012','12300402','12300403','12300501','12300802','11200101','11200102','11200201','11000204','11000206','11100802','11101001','11100603','11100608','15400204','15400406','2540040114','15400604','15200105','18101601','2820010104','2820010204','482001011','482001021','18200401','18200402','18200403','10201208','10201301','12202403','10100302','15100403','392002014','492006012','19200802','15600401','2920010111','2760020607','376002068','476002056','12700102','15700102','15700107','15700202','12202701','12700201','2570040119','10201216','13300103','13200206','13200207','13200301','13200304','13300301','13300401','13300501','2810030101','381003015','68127A','2810050101','381006015','18100901','18101302','18000104','18101903','18102003','18800201','18800401','2880040103','488003031','19500112','16200201','16200302','10201215','16300201','19300115','393001096','19300202','19300307','10401304','10400601','10400701','15300103','15300201','15000103','15000203','2510030619','15100404','13100201','13101401','13100502','13100702','13100801','19100104','2910010109','19100207','19101401','19200202','19200209','19400201','19100607','19400203','14100204','14100801','14100901','342002012','14200303','14200306','11300401','452004016','17100901','471009053','17200105','17300101','473001013','2740020107','17500107','475001073','2760030207','37600302S','17700301','393001026','12801101','17100801','17500101','17500103','475001043','17600305','17800203','2930010313','493003036','12100101','12200301','12200501','12200603','12201001','12201803','12800511','2230010111','12300201','12300401','12300406','12300407','2110020112','311003011','311005011','19101101','2540020314','15400405','354005014','15400902','18101501','2810150101','18101701','482001041','17800100','17800102','10200101','10202302','10202304','10200402','10202201','12202302','15100101','2510020119','15100302','15100305','2560010111','2920020211','19200702','15600104','19200401','19200803','15300109','15700104','15700601','15700405','15700407','10200204','10201205','13200105','2320010116','13300102','2330010116','13200203','13300203','2810010102','18100301','481005011','481010011','18000101','488001011','388002015','2950010310','16200803','10201213','16100201','19300109','2930010713','19300407','2930040513','2930011013','393001126','19300306','2930030513','10400303','10401305','10400305','10400501','10401501','10401503','10400602','10400702','2520010120','15200201','352002033','2760020107','10200502','10200507','10202401','14000101','2310020315','13100403','13100407','13100604','13100701','13100405','15600105','19200901','19400113','394001011','19100205','19100604','19100801','19101301','14100201','14200307','14200311','11300108','11300302','413003015','14000202','14000203','14000403','14100701','14200501','14200502','15200501','17300201','17300202','474002093','87527A','376001036','17700202','2930010213','17100202','17400502','2740050107','17500102','2760030307','2760030507','12200101','12202902','12202905','12203205','12800101','12800407','328005011','12300101','2230060111','11100201','11000202','310002011','310002081','11100605','2110060212','11100602','11100404','15400101','15400208','15400505','15400901','352001063','17700102','17800101','2810140102','2810160102','2820010304','2820040104','10201106','10201304','10202602','10200703','10200404','10200405','10200601','10202202','2510020319','15100304','19200102','392005024','392006014','19200105','19200110','2920010711','15300106','17600204','476002046','12700103','12700104','2370010118','15700105','2570020118','15700301','15700402','10200201','13200107','13200108','332001014','13300101','13200202','381004015','18100501','381005015','2810130201','381013025','18102001','18102501','388006015','48800302S','16200501','19300108','393001086','10400302','2040030108','10401502','10401401','10401702','2520010220','452001076','15200202','15200205','15300102','15300105','17600201','15000206','15000207','15000801','2520010420','17600203','10200501','10200509','15100309','10200307','13100802','13100805','13100301','331004034','13100602','13100603','13100803','13101304','19100208','19200201','19100302','19100601','491006025','19200501','14200301','2420030317','14200401','14200405','11300101','11300107','11300201','14000301','14000401','14100104','15200401','17100903','17100905','471009043','17200106','472001063','17400209','37400209S','17600106','17600307','2760030107','393004026','17100201','17100401','17100402','17400505','376003028','12200202','12200402','12203201','12203402','12201702','12201805','12201807','12203105','12800303','12300701','11100301','11100402','2110040212','15000301','2540020114','2540040314','352001053','18200105','18200302','2820050204','10200105','10201305','10202003','10202301','10201801','12202401','10100301','15100203','2510020219','15100501','15100601','19200106','392001034','492001022','2920010211','392001044','19200302','19200601','2760020407','376002048','2760020507','12700202','2570040218','13200102','13200501','2330010216','13700103','2320020116','13200401','13300302','18101001','18101901','388001015','488003011','19500105','19500111','19500201','2950020110','16200101','16200602','16100101','16100401','16200502','16200701','19300408','19300111','19300113','10400306','10401201','10401601','10401602','15200101','15200104','452001116','352003013','376002038','10200308','13100205','13100207','13100804','13100402','13100406','13100408','13100501','13100504','13100902','13100903','13100906','456001014','2910010209','391001014','19100503','391005021','19100108','19100204','19101402','19400108','2940010309','11300204','14100202','14100203','14200203','342002022','342004012','11300103','11300106','11300202','11300203','14200305','17100101','17100902','17400203','17400204','17400207','17400208','474006013','475001063','476003053','477001041','18800102','488001021','2930040113','17100204','17100601','17300203','2730020307','17400506','476003033','19300106','12200201','12200601','12200801','12203203','12203503','12800509','12201706','12201710','12202001','12202003','12800402','12800403','428005015','12800901','12800902','323002014','410002015','11100504','11200302','15400207','15400301','15400302','15400801','15400503','15400504','2520010504','17700201','18101602','482001051','18200601','17800103','10200103','10200903','10202601','10200902','10202701','12202304','12202402','2510010119','15100204','15100602','15600103','2920020111','19200107','19200111','11300501','15700101','15700106','15700108','2570010118','15700201','18700101','15700404','15700408','13200109','13300104','10202801','13200204','13200208','13200303','481004011','2810130101','481013021','2880020103','488003021','18800303','48800303S','19500104','19500109','19500113','10400402','19300405','2930040413','393001116','2040030208','2040050108','10400703','15200108','452001106','476002016','15000102','15000501','350007013','15200301','2760020307','476002026','10200513','10200503','10200305','13100703','13101003','13100601','19100105','19100401','19100502','492002014','19200504','19400401','2910050109','19100603','19200203','11300105','14200103','14200402','14200403','11300301','11300402','14000201','2400040117','14100102','14100108','14100111','14100902','17100103','17100702','17100904','2710090507','471009033','47400101S','37400208S','17600101','17700105','19300104','393001046','17100503','17100806','2730010207','473001023','475001033','17600304','19300502','12200901','12203204','12203502','12800503','12801001','12201703','12201802','12202601','12202801','12203102','12800404','12800505','12800903','323001014','12300404','12300801','11000203','410002085','311005021','11100502','19101201','15400209','15400502','2540050114','15000401','15400603','15400701','2770020105','381015015','18101604','18200102','10201103','10201201','10201209','10201302','10201501','10202002','10200701','10200702','10202101','12202405','15100603','392001064','15300107','15300108','15700103','15700401','15700403','15700501','10200202','13200106','2810060101','18101902','18102004','18102101','18102102','18102401','18800301','48800301S','2950010210','2950020210','16200401','16200801','16100501','393004056','2930011213','393003076','10400301','10400401','10400802','2520020321','15300104','15000105','15200303','17600202','10200309','331002024','13101501','13101502','456001054','19100202','19200208','2910020509','19400104','19400106','14100802','2420020317','14000205','14100105','14100110','14100601','17100701','17400101','17400301','17500105','17500106','375001057','67527A','17600103','17600308','476005013','48800102S','2930040213','17100802','471008053','17200103','17500104','2760030407','19300103','19300301','19300302','19300304','12200502','12201401','12203001','12203501','12800201','12800512','12201705','12201707','12201801','12202501','12203107','12800405','12800504','12800801','12300405','12300803','11000101','11000201','11000207','11000209','2120030212','11200501','19101202','15400202','15400408','15200106','17700103','2770010205','2770010305','477001031','18101603','18200301','18200502','10200102','10200104','10201102','10201105','10201202','10201211','10201212','10201303','10202001','10200901','10200401','15100201','19200103','392001074','19200301','19200804','476002066','2570030118','10201210','13200110','2810040101','18100601','18101202','18800101','2880010103','18800601','18800501','19500106','19500202','19500301']

# Fonctions Utiles




In [7]:

def ocr_ancien_controle_legalite(pdf):
    
    detect_controle = ''
    date = ''
    texte = []
        
    page = pdf.load_page(0)
    tp = page.get_images()
    
    #print(len(tp))
    for j in range(len(tp)):
        #print(j)
        xref = tp[j][0]
        image = pdf.extract_image(xref)
        #print (image)
        
        with TemporaryDirectory() as tempdir:
            # Create a temporary directory to hold our temporary images
            
            filename = f"{tempdir}\controle_{xref}.{image['ext']}"
            
            imgout = open(filename, "wb")
            imgout.write(image["image"])
            imgout.close()
            
            #tempimg = Image.open(filename)
            
            tempimg = cv2.imread(filename)
            tempimg = cv2.resize(tempimg, (2000, 800))
            
            #blur = cv2.bilateralFilter(tempimg,9,75,75)
            # kernel = np.ones((5,5),np.uint8)
            # Si on a du bruit autour opening = cv.morphologyEx(img, cv.MORPH_OPEN, kernel)
            # Si on a des trous closing = cv.morphologyEx(img, cv.MORPH_CLOSE, kernel)
            # dilatation dilation = cv.dilate(img,kernel,iterations = 1)
            
            gray = cv2.cvtColor(tempimg, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (3,3), 0)
            thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
            # Morph open to remove noise and invert image
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
            opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
            invert = 255 - opening
            
            #cv2.imshow('Result', invert)
            #plt.imshow(invert)
            #plt.show()
            
            # Perform text extraction
            data = pytesseract.image_to_string(invert, lang='fra', config='--psm 4')
            #data contient plusieurs lignes de texte
            #print("DATA : ", data)
            #d = pytesseract.image_to_pdf_or_hocr(tempimg, lang='fra', extension='hocr').decode('utf-8')
            
            regex_date = r"(\d{2})\s*/\s*(\d{2})\s*/\s*(\d{4})"
            match_controle = re.findall(regex_date, data)
            if (len(match_controle)):
                date = match_controle[0]
                # Convertir en YYYY-MM-DD
                detect_controle = date[2] + "-" + date[1] + "-" + date[0]
                print("---")
                print("Date de contrôle trouvée : ", detect_controle)
                print("---")
            else:
                print("---")
                print("Erreur rencontrée: le match est vide : ", data)
                print("---")
            
            #try:
            #    text = pytesseract.image_to_string(data, lang='fra')
            #    tempimg.close()
            #    print("TEXT : ", text)
            #    #print(len(text))
            #    if (text.find('publication') != -1):
            #        regex_date = r"(\d{2})\s*/\s*(\d{2})\s*/\s*(\d{4})"
            #        match_controle = re.findall(regex_date, text)
            #        if (len(match_controle)):
            #            # print("DATE DETECTEE : " + str(date_seance))
            #            date = match_controle[0]
            #            # Convertir en YYYY-MM-DD
            #            detect_controle = date[2] + "-" + date[1] + "-" + date[0]
            #        break
            #    else:
            #        print("publication pas trouvée")
            #except:
            #    print("Erreur rencontrée")
            #    continue
    
    return detect_controle



In [8]:
#Différentes Fonctions de traitements des textes

def traiter_rapport(extraction_blocks):
    blocks = []
    fin_rapport = False
    for item in extraction_blocks:
            #item est un tuple à éléments
            a,b,c,d,e,f,g = item    
            if (any(x in e for x in ["EXPOSÉ DES MOTIFS" , "CONSEIL RÉGIONAL D’ÎLE-DE-FRANCE"])):
                pass
            elif ("Je vous prie de bien vouloir en délibérer" in e):
                fin_rapport = True
                break
            else:
                blocks.append(e.replace("\n", " ").replace("  ", " "))
    return (blocks, fin_rapport)

def traiter_deliberation(extraction_blocks):
    blocks = []
    fin_rapport = False
    for item in extraction_blocks:
            #item est un tuple à éléments
            a,b,c,d,e,f,g = item    
            if (any(x in e for x in ["DÉLIBÉRATION N°CP","DÉLIBÉRATION N°CR","DELIBERATION N°CP","DELIBERATION N°CR"])):
                pass
            elif ("Acte rendu exécutoire le" in e):
                fin_rapport = True
                break
            else:
                blocks.append(e.replace("\n", " ").replace("  ", " "))
    return (blocks, fin_rapport)

def extract_sommaire_annexes_rapport(dictionary_elements):
    """ Détecte si la page courante correspond au sommaire de la délibération
    Le cas échéant, la fonction extrait le sommaire et la liste des annexes
    Args: TextPage extraite sous forme de dictionnaire (extractDICT())
    """
    sommaire = []
    indicateur_sommaire = False
    indicateur_annexes_deliberation = False
    for block in dictionary_elements['blocks']:
        for line in block['lines']:
            for span in line['spans']:
                if indicateur_sommaire == True and indicateur_annexes_deliberation == True:
                    if re.match("Annexe", span['text']) is not None:
                        sommaire.append(span['text'])
                    #display(span['text'])
                    #print(result)
                if span['text'] == "ANNEXES À LA DÉLIBÉRATION":
                    indicateur_annexes_deliberation = True
                if span['text'] == "Sommaire":
                    indicateur_sommaire = True
                
    return sommaire

def sommaire_to_html(sommaire_list: list) -> str:
    """ Transforme un sommaire_list en liste html
    """
    if (not len(sommaire_list)):
        return ""
    
    sommaire_html = "<ul>"
    for item in sommaire_list:
        sommaire_html += "<li>" + item + "</li>"
    sommaire_html += "</ul>"
    return sommaire_html

def merge_blocks(block_1:dict=None, block_2:dict=None) -> dict:
    """ Créer un block issu des deux précédents passés en paramètres
    """
    if block_1 == None:
        return block_2
    if block_2 == None:
        return block_1
    a, b, c, d = block_1['bbox']
    c, d, e, f = block_2['bbox']
    bbox = (a, b, e, f)
    block = {
        'number': 0,
        'type': 0,
        'bbox': bbox,
        'lines': []
        }
    for line in block_1['lines']:
        block['lines'].append(line)
    for line in block_2['lines']:
        block['lines'].append(line)
    return block

##
## TODO auditer fonction merge_all_blocks
##

def merge_all_blocks(dictionary_elements:dict=None) -> dict:
    """Effectue quelques contrôles pour regrouper les blocs séparés
    Par exemple si la dernière ligne du précédent bloc ne finit pas par un point
    et que la première ligne du bloc courant ne commence pas par une majuscule,
    on regroue le bloc courant avec le précédent.
    
    Args: dictionary_elements, dictionnaire obtenu à partir de la méthode extractDICT()
    """
    blocks_list = []
    mark_list = []
    for block in dictionary_elements['blocks']:
        blocks_list.append(block)
    #display(len(list_blocks))
    new_block_dict = dictionary_elements
    new_block_dict['blocks'] = []
    #for block in dictionary_elements['blocks']:
    for index, block in enumerate(blocks_list):
        
        if index >= 0 and index < len(blocks_list):
            
            # Pour le bloc en cours, on initialise la valeur de la première et de la dernière "span"
            first_span = ""
            for line in block['lines']:
                last_span = ""
                for span in line['spans']:
                    if first_span == "" and span['text'] != "":
                        first_span = span['text']
                    if (span['text'] != ""):
                        last_span = span['text']
            #display(first_span + " // " + last_span)
            block_specs = {"first": first_span,  "last": last_span}
            if (first_span == "" or last_span == ""):
                print("Problème merge_all_blocks: first span ou last span du bloc vide. First span: ", first_span, " <> Last span: ", last_span)
            if (first_span == "" and last_span == ""):
                print("Bloc vide: on passe totalement")
            mark_list.append(block_specs)
            
            if index > 0:
                # Si premier caractère du bloc courant est minuscule + dernier caractère du précédent n'est pas un point
                if first_span[0].islower() and mark_list[index-1]['last'][-1] not in ('.', '!', '?'):
                    new_block_dict['blocks'].append(merge_blocks(blocks_list[index-1], block))
                    if (index == len(blocks_list)):
                        print("Problème on dépasse 1")
                else:
                    new_block_dict['blocks'].append(blocks_list[index-1])
                    if (index == len(blocks_list)-1):
                        print("Dernier bloc qu'on ajoute")
                        texte_du_bloc = ""
                        for line in block['lines']:
                            texte_du_bloc += " //LIGNE// "
                            for span in line['spans']:
                                texte_du_bloc += " " + span['text']
                        print(texte_du_bloc)
                        print("---")
                        #print(str(block))
                        new_block_dict['blocks'].append(block)
                    if (index == len(blocks_list)):
                        print("Problème on dépasse 2")
                # Test if last_span-1 
            
            # test if last_span ends with a punctuation mark
        
        if (index >= len(blocks_list)):
            print("Erreur avec la fonction enumerate: l'index présent dans merge_all_blocks est supérieur ou égal à la longueur de la liste de blocs")
    
    return new_block_dict


def dict_to_html(dictionary_elements: dict) -> str:
    block_text = ""
    for block in dictionary_elements['blocks']:
        #print(block)
        
        if (dict_block_to_text(block).replace(" ","") == ""):
            continue
        
        # TODO Pour les visas, préférer les ul / li aux paragraphes classiques
        
        block_text += "<p>"
        for line in block['lines']:
            line_text = ""
            for span in line['spans']:
                #print(span)
                if "Bold" in span['font'] or "bold" in span['font']:
                    if "Italic" in span['font'] or "italic" in span['font']:
                        texte_a_inclure = "<b><i>" + span['text'] + "</i></b>"
                    else:
                        texte_a_inclure = "<b>" + span['text'] + "</b>"
                else :
                    texte_a_inclure = span['text']
                line_text += " " + texte_a_inclure
                #print(span)
            block_text += line_text
            #line_text += " STOP "
            #print(line_text)
        block_text = re.sub(' +', ' ', block_text.strip())
        block_text += "</p>"
    return block_text

def append_dict_as_row(file_name, dict_of_elem, field_names):
    # Open file in append mode
    with open(file_name, 'a+', newline='', encoding="utf-8") as write_obj:
        # Create a writer object from csv module
        dict_writer = DictWriter(write_obj, fieldnames=field_names)
        # Add dictionary as wor in the csv
        dict_writer.writerow(dict_of_elem)

def dict_block_to_text(dict_block) -> str:
    text = ""
    for line in dict_block['lines']:
        for span in line['spans']:
            text += span['text']
    return text

def normalise_identifiant_cpcr(ide: str, verbose = False):
    ide = ide.replace(" ", "").rstrip('\r\n')
    # Si format 20xx-XYZ, on ajoute un zéro pour normaliser si seulement deux chiffres à la fin
    identifiant_deux_chiffres = re.search(r"(?P<type>[CPR]{2})\s?(?P<annee>[0-9]{4})\s?-\s?(?P<marqueur>[C]?)\s?(?P<numero>[0-9]{2})\D", ide)
    if (identifiant_deux_chiffres is not None):
        ide = identifiant_deux_chiffres.group('type') + identifiant_deux_chiffres.group('annee') + "-" + identifiant_deux_chiffres.group('marqueur') + "0" + identifiant_deux_chiffres.group('numero')
        if (verbose) : print("TROUVE identifiant avec seulement 2 chiffres en format post 2016, zéro ajouté :  ", ide)
    else:
        if (verbose) : print("Identifiant trouvé a plus de 2 chiffres à la fin si format post 2016 :  ", ide)
    
    identifiant_complet = re.search(r"(?P<type>[CPR]{2})\s?(?P<annee>[0-9]{4})\s?[-\u00AD]\s?(?P<marqueur>[C]?)\s?(?P<numero>[0-9]{3})", ide)
    identifiant_ante2017cp = re.search(r"(?P<type>CP)\s*(?P<annee>1[0-9])\s*[-\u00AD]\s*(?P<numero>[0-9]{2,3})", ide)
    identifiant_ante2017cr = re.search(r"(?P<type>CR)\s*(?P<numero>[0-9]{2,3})\s*[-\u00AD]\s*(?P<annee>1[0-9])", ide)
    if (identifiant_complet is not None):
        ide = identifiant_complet.group('type') + identifiant_complet.group('annee') + "-" + identifiant_complet.group('marqueur') + identifiant_complet.group('numero')
    elif (identifiant_ante2017cp is not None):
        # Si numero ne fait que deux chiffres
        if (len(identifiant_ante2017cp.group('numero')) == 2):
            numero = "0" + identifiant_ante2017cp.group('numero')
        else:
            numero = identifiant_ante2017cp.group('numero')
        ide = identifiant_ante2017cp.group('type') + identifiant_ante2017cp.group('annee') + "-" + numero
        print("IDENTIFIANT ante 2017 CP")
    elif (identifiant_ante2017cr is not None):
        # Si numero ne fait que deux chiffres
        if (len(identifiant_ante2017cr.group('numero')) == 2):
            numero = "0" + identifiant_ante2017cr.group('numero')
        else:
            numero = identifiant_ante2017cr.group('numero')
        ide = identifiant_ante2017cr.group('type') + numero + "-" + identifiant_ante2017cr.group('annee')
        print("IDENTIFIANT ante 2017 CR")
    else:
        print("ERREUR dans l'identifiant (non reconnu par la regex) ", ide)
    return ide.strip().rstrip('\r\n').replace(" ", "").replace("\u00AD","-").replace("N°","")

In [9]:
def pikepdf_decrypte_fichier(filename: str) -> str:
    """Permet de passer un fichier à la moulinette pikepdf.
    Cela permet notamment de passer des fichiers cryptés
    et dans des versions trop récentes pour camelot (qui utilise PyPDF2)"""
    with Pdf.open(filename, password="") as pdf:
        #sauvegarde du fichier sous un nouveau nom: 
        pdf.save(filename + str(1))
        # on ferme l'ancien fichier
        pdf.close()
    # on supprime l'ancier fichier
    os.remove(filename)
    # on renomme l'ancien fichier
    os.rename(filename + str(1), filename)
    #os.close(filename)
    return filename


In [10]:
def date_du_jour():
    date_publication = datetime.today().strftime('%Y-%m-%d')
    return date_publication

def thematique_from_commission(commission: str) -> str:
    """
    EOLE
    ADMIN GÉNÉRALE
    AGRI
    CULTURE
    DEV ÉCO
    EGALITÉ
    ENSSUP RECH.
    ENVI.
    ESS
    FAMILLE
    FINANCES FDS EUR.
    EMPLOI
    INTERNATION. EUROPE
    LOGEMENT AMÉNAG.
    LYCÉES
    PROMESSE RÉP.
    RÉFOR. TERRIT.
    RÈGLEMENT
    RURALITÉ
    SANTÉ
    SÉCURITÉ
    SPORT
    TOURISME
    TRANSPORTS
    "lycées"
    "finances"
    "administration"
    "culture"
    sports
    économi
    logement
    aménagement
    environnement
    emploi
    enseignement supérieur
    recherche
    santé
    famille
    jeunesse
    international
    ruralité
    tourisme
    agriculture
    sécurité
    réforme
    formation

    """
    if (commission == ""):
        return "Institution"
    elif ("lycées" in commission):
        return "Lycées"
    elif ("transports" in commission or "mobilité" in commission):
        return "Transports et mobilités"
    elif ("finances" in commission):
        return "Finances et fonds européens"
    elif ("administration" in commission):
        return "Administration générale"
    elif ("sports" in commission):
        return "Sports, Jeunesse, Vie associative"
    elif ("économi" in commission):
        return "Economie et Emploi"
    elif ("logement" in commission):
        return "Logement et Aménagement"
    elif ("aménagement" in commission):
        return "Logement et Aménagement"
    elif ("environnement" in commission):
        return "Environnement"
    elif ("emploi" in commission):
        return "Formation professionnelle et Emploi"
    elif ("enseignement supérieur" in commission or "recherche" in commission):
        return "Enseignement supérieur et Recherche"
    elif ("santé" in commission):
        return "Santé"
    elif ("famille" in commission):
        return "Famille, Action sociale, Handicap"
    elif ("jeunesse" in commission):
        return "Jeunesse, Promesse républicaine et Insertion professionnelle"
    elif ("international" in commission):
        return "Europe et International"
    elif ("ruralité" in commission):
        return "Ruralité, Commerce et Artisanat"
    elif ("tourisme" in commission):
        return "Tourisme"
    elif ("agriculture" in commission):
        return "Agriculture et Alimentation"
    elif ("culture" in commission):
        return "Culture"
    elif ("sécurité" in commission):
        return "Sécurité"
    elif ("réforme" in commission):
        return "Institution"
    elif ("formation" in commission):
        return "Formation professionnelle et Emploi"
    else:
        return ""

def numero_from_identifiant(identifiant: str) -> str:
    taille = len(identifiant)
    if (taille < 3):
        return identifiant
    return identifiant[0:2] + " " + re.sub(r'.', '', identifiant, count = 2)

# Nettoyage html

In [12]:

# @TODO: créer fonction de nettoyage HTML comme pour les arrêtés
# Retirer les paragraphes tels que:
#2022-12-09 22:15:12
#ID : 075-237500079-20221209-CR_2022_079-AR



# Traite un fichier

In [14]:


def extract_id_titre_rapport(dictionary_elements: dict, verbose = False):
    ide = ""
    titre = ""
    for block in dictionary_elements['blocks']:
        
        x0, y0, x1, y1 = block['bbox']
        if (y0 > 300 and y0 < 600):
            for line in block['lines']:
                for span in line['spans']:
                    titre += " " + span['text']
            
        for line in block['lines']:
            line_text = ''
            for span in line['spans']:
                line_text += " " + span['text']
            identifiant = re.findall(r"(?:C[PR]{1}\s*20[0-9]{2}\s*-\s*C?[0-9]{2,3}|CR\s*[0-9]{2,3}\s*-\s*1[0-9]|CP\s*1[0-9]\s*-\s*[0-9]{2,3})", line_text) #tient compte rapport COVID CP2021-C01
            if (len(identifiant)):
                ide = normalise_identifiant_cpcr(identifiant[0])
                # OK print(ide)
                
    return (ide, titre.strip())

def extract_expose_rapport(dictionary_elements: dict, tables_camelot, verbose = False):
    
    last_block = 0
    
    # DEBUT : boucle2 parcourir les tableaux trouvés dans la page courante
    if (tables_camelot != None):
        for t in range(tables_camelot.n):

            tableau_place = False
            table_html = ((tables_camelot[t]).df.to_html(index=False,header=False))
            table_html = table_html.replace('\xa0',' ')
            soup = BeautifulSoup(table_html,"html.parser")

            # On recupere le texte brut du tableaux t
            texte_table = soup.get_text()
            texte_table = texte_table.strip()

            # Pour obtenir le nombre de ligne dans le tableau, on a observé que les lignes sont separées par'\n\n\n'
            number_of_lines = (len(re.findall('\n\n\n', texte_table))) + 1
            texte_table = texte_table.replace('\n\n\n','\n ')

            # DEBUT boucle3: parcourir_tous_les_paragraphes_page
            for k in range(last_block, len(dictionary_elements['blocks'])):
                # Si le ploque commence par '\n' on effectue aucun comparaison
                block_text = dict_block_to_text(dictionary_elements['blocks'][k]).strip()
                if (block_text == '\n'):
                    continue
                a = texte_table.replace("\n","").replace(" ","").find(block_text.replace("\n","").replace(" ",""))
                # Condition3_1: Verife si le tableau t doit etre place dans le paragrahe en question
                if( a>-1 and tableau_place==False):
                    # cela signifie qu'on a trouve le texte du tableau dans la ligne
                    tableau_place = True
                    numero_block = k #dictionary_elements['blocks'][k][5]
                    # print('block:'+str(block_list[k][5]))
                    # On incrémente le derniere block pour faciliter l'ajout d'un autre tableau au besoin
                    last_block = k
                    #print(numero_block)
                    #Permet d'avoir le nombre de colonne d'une table
                    longueur_table = len(dictionary_elements['blocks'][numero_block]['lines'])

                    # Les boucles suivantes (imbriquées) permettent de parcourir les lignes de textes contenues
                    # dans dans les paragraphes, ce qui correspond au lignes et colonnes des tables

                    # DEBUT boucle4: parcourir_tous_les paragraphe_par blocks_
                    for j in range(number_of_lines):
                        #DEBUT boucle5: Permet de parcourir chaque lignes des blocks
                        for p in range(0, longueur_table):
                            if (j == 0 and p==0):
                                # Ajoute le tableau
                                # dictionary_elements['blocks'][numero_block] = { "type": 0, "bbox": (0, 0, 0, 0), "lines": [] }
                                """
                                "lines": ({
                                    "wmode": 0,
                                    "dir": (1.0, 0.0),
                                    "bbox": (50.0, 88.17500305175781, 166.1709747314453, 103.28900146484375),
                                    "spans": ({
                                        "size": 11.0,
                                        "flags": 0,
                                        "font": "Helvetica",
                                        "color": 0,
                                        "origin": (50.0, 100.0),
                                        "text": "Some text on first page.",
                                        "bbox": (50.0, 88.17500305175781, 166.1709747314453, 103.28900146484375)
                                    })
                                })

                                """
                                dictionary_elements['blocks'][numero_block]['lines'][0]['spans'][0]['text'] = '<div class="table-camelot">' + table_html + '</div>'

                            elif((numero_block+j) < len(dictionary_elements['blocks'])):
                                dictionary_elements['blocks'][numero_block+j] = { "type": 0, "bbox": (0, 0, 0, 0), "lines": [] }
                                # dictionary_elements['blocks'][numero_block+j]['lines'][p]['spans'][0]['text'] = ""
                        # FIN boucle5
                    # FIN boucle4
                # FIN if Condition3_1
            # FIN boucle3: parcourir_tous_les_paragraphes_page_i       

        # FIN : boucle2 parcourir les tableaux trouvés dans la page courante
    
    limite_inf = 50
    limite_sup = 800
    fin = False
    
    dictionnaire_expurge = {'blocks': []}
    
    for block in dictionary_elements['blocks']:

        x0, y0, x1, y1 = block['bbox']
        
        texte = ""
        for line in block['lines']:
            for span in line['spans']:
                texte += " " + span['text']
        
        if ("EXPOSÉ DES MOTIFS" in texte or "EXPOSE DES MOTIFS" in texte):
            # On passe au paragraphe qui suite "Exposé des motifs"
            print("EXTRACTION ", texte)
            continue
            
        regex_paragraphe_date_seule = r"^\s*(?:\d{4}-\d{2}-\d{2}|\d{2}\/\d{2}\/\d{4})\s+\d{2}:\d{2}:\d{2}\s*$"
        if (re.search(regex_paragraphe_date_seule, texte) is not None):
            # On passe au paragraphe qui suit
            print("EXTRACTION ", texte)
            continue
            
        if ("ANNEXE AU RAPPORT" in texte):
            # On arrête la lecture du rapport avant les annexes
            fin = True
            break
        if ("PROJET DE DÉLIBÉRATION" in texte or "PROJET DE DELIBERATION" in texte):
            # On arrête la lecture du rapport avant le projet de délibération
            fin = True
            break
        
        if (y0 > limite_inf and y1 < limite_sup):
            dictionnaire_expurge['blocks'].append(block)
        
        """
        # CONTROLE SUR LES SUPPRESSIONS LIEES AUX MARGES
        if (y0 > limite_inf and y1 < limite_sup):
            print("y0 = " + str(y0) + " et y1 = " + str(y1) + " :" + texte)
        else:
            print(" ")
            print("Supprimé: " + texte)
            print(" ")
        """
    
    dictionnaire_expurge = merge_all_blocks(dictionnaire_expurge)
    expose_part = dict_to_html(dictionnaire_expurge)
    # print(expose_part)
    return (expose_part, fin)

def extract_id_date_delib(text_string: str, verbose = False):
    """
    @TODO Transformer avec regex
    DELIBERATION N° CP 2017-017
    N° CR N° 2017-101
    Du 27 janvier 2017
    """
    
    #regex_id_date_delib = r"D(?:E|É)LIB(?:E|É)RATION\s*(?:N°)?\s*(?P<id>(?:CP|CR)\s*(?:N°\s*)?(?:20\d{2}\s?[-\u00AD]\s?C?\d{2,3}|\d{2,3}\s?[-\u00AD]\s?1\d))(?:\s*\n\s*(?:DU|Du)\s*)?(?P<date>\d{1,2}\s*[ADFJMNOSadfjmnos][\wûÛ]*\s*[\d]{4})"
    # Prend en compte CP16-123 et CR123-16
    regex_id_date_delib = r"D(?:E|É)LIB(?:E|É)RATION\s*(?:N°)?\s*(?P<id>CP\s*(?:N°\s*)?1\d\s*[-\u00AD]\s*\d{2,3}|CR\s*(?:N°\s*)?\d{2,3}\s*[-\u00AD]\s*1\d|(?:CP|CR)\s*(?:N°\s*)?20\d{2}\s*[-\u00AD]\s*C?\d{2,3})(?:\s*\n\s*(?:DU|Du)\s*)?(?P<date>\d{1,2}(?:er)?\s*[ADFJMNOSadfjmnos][\wûÛ]*\s*[\d]{4})"
    
    cherche_id_date = re.search(regex_id_date_delib, text_string)
    
    if (cherche_id_date is not None):
        if (cherche_id_date.group('id') is not None):
            identifiant = cherche_id_date.group('id').strip().rstrip('\r\n').replace(" ", "").replace("\u00AD","-").replace("N°","")
            # Remplacer les tirets non conventionnels
            # [\u002D\u058A\u05BE\u1400\u1806\u2010-\u2015\u2E17\u2E1A\u2E3A\u2E3B\u2E40\u301C\u3030\u30A0\uFE31\uFE32\uFE58\uFE63\uFF0D]
        if (cherche_id_date.group('date') is not None):
            date = cherche_id_date.group('date').strip().rstrip('\r\n')
        else:
            print("========================")
            print("--- Date non trouvée ---")
            print("TEXT SRING :::: ", text_string)
            print("========================")
            print("CHERCHE ID DATE :::: ", cherche_id_date)
            print("========================")
            regex_date_delib_secours = r"(?:(?:DU|Du)\s*)?(?P<date>\d{1,2}(?:er)?\s+[ADFJMNOSadfjmnos][\wûÛ]{3,10}\s+[\d]{4})?"
            cherche_date_secours = re.search(regex_date_delib_secours, text_string)
            if (cherche_date_secours is not None and cherche_date_secours.group('date') is not None):
                date = cherche_date_secours.group('date').strip().rstrip('\r\n')
                print(" ======= DATE SECOURS TROUVEE ======== ")
                print("Date trouvée : ", date)
                print("========================")
            else:
                print(" ======= BLOCAGE COMPLET CAR PAS DE DATE ======== ")
    else:
        print("========================")
        print("--- cherche_id_date == None ---")
        print("TEXT SRING :::: ", text_string)
        print("========================")
        # On essaie de trouver
        # RAPPORT N° CR 2017-188 
        # PROJET DE DELIBERATION 
        # DU 23 NOVEMBRE 2017
        regex_id_date_delib_secours = r"(?:N°)?\s*(?P<id>CP\s*(?:N°\s*)?1\d\s*[-\u00AD]\s*\d{2,3}|CR\s*(?:N°\s*)?\d{2,3}\s*[-\u00AD]\s*1\d|(?:CP|CR)\s*(?:N°\s*)?20\d{2}\s*[-\u00AD]\s*C?\d{2,3}).*(?:\n)?.*(?:\n\s*(?:DU|Du)\s*)?(?P<date>\d{1,2}(?:er)?\s*[ADFJMNOSadfjmnos][\wûÛ]*\s*[\d]{4})"
        cherche_id_date_secours = re.search(regex_id_date_delib_secours, text_string)
        if (cherche_id_date_secours is not None and cherche_id_date_secours.group('date') is not None):
            identifiant = cherche_id_date_secours.group('id').strip().rstrip('\r\n').replace(" ", "").replace("\u00AD","-").replace("N°","")
            date = cherche_id_date_secours.group('date').strip().rstrip('\r\n')
        else:
            print(" ======= BLOCAGE COMPLET CAR RIEN IDENTIFIE ======== ")
        
    
    # TODO à nettoyer supprimer lorsque plus nécessaire
    #debut = text_string.find('DÉLIBÉRATION N°')
    #fin = text_string.find('DU')
    #if debut != -1 and identifiant == "":
    #    identifiant = text_string[debut+15:fin-27].replace(" ", "").rstrip('\r\n')
    #    # TODO Clarifier la méthode +15 -27

    #date_seance = re.findall(r"[\d]{1,2} [ADFJMNOSadfjmnos][\wûÛ]* [\d]{4}", text_string)
    #date = date_seance[0]
    # Convertir en YYYY-MM-DD
    annee = date[-4:]
    
    if (re.search(r"jan", date, re.IGNORECASE)):
        mois = "01"
    elif (re.search(r"vrier", date, re.IGNORECASE)):
        mois = "02"
    elif (re.search(r"mars", date, re.IGNORECASE)):
        mois = "03"
    elif (re.search(r"avril", date, re.IGNORECASE)):
        mois = "04"
    elif (re.search(r"juin", date, re.IGNORECASE)):
        mois = "06"
    elif (re.search(r"juil", date, re.IGNORECASE)):
        mois = "07"
    elif (re.search(r"ao", date, re.IGNORECASE)):
        mois = "08"
    elif (re.search(r"sep", date, re.IGNORECASE)):
        mois = "09"
    elif (re.search(r"oct", date, re.IGNORECASE)):
        mois = "10"
    elif (re.search(r"nov", date, re.IGNORECASE)):
        mois = "11"
    elif (re.search(r"cemb", date, re.IGNORECASE)):
        mois = "12"
    elif (re.search(r"mai", date, re.IGNORECASE)):
        mois = "05"
    else:
        mois = "08"
    
    jour = date[0:2].strip().rstrip("er")
    if (len(jour) == 1):
        jour = "0"+ jour
        
    date = annee + "-" + mois + "-" + jour
    
    return (normalise_identifiant_cpcr(identifiant), date)

def extract_projet_delib_rapport(dictionary_elements: dict, verbose = False):
    
    limite_inf = 50
    limite_sup = 800
    debut = False
    fin = False
    projet_delib_part = ""
    date_presentation_part = ""
    
    dictionnaire_expurge = {'blocks': []}
    
    for block in dictionary_elements['blocks']:

        x0, y0, x1, y1 = block['bbox']
        
        texte = ""
        for line in block['lines']:
            for span in line['spans']:
                texte += " " + span['text']
        
        # Si on détecte en regex "PROJET DE DÉLIBÉRATION"
        cherche_debut = re.findall(r"PROJET DE DÉLIBÉRATION", texte)
        if (len(cherche_debut)):
            # print("DEBUT : " + texte)
            debut = True
        
        # On détecte la date
        date_seance = re.findall(r"DU\s*([\d]{1,2})(?:er)?\s*([ADFJMNOSadfjmnos][\wûÛ]{2,10})\s*([\d]{4})", texte)
        if (len(date_seance)):
            # print("DATE DETECTEE : " + str(date_seance))
            date = date_seance[0]
            # Convertir en YYYY-MM-DD
            annee = date[2]
            
            if (re.search(r"jan", date[1], re.IGNORECASE)):
                mois = "01"
            elif (re.search(r"vrier", date[1], re.IGNORECASE)):
                mois = "02"
            elif (re.search(r"mars", date[1], re.IGNORECASE)):
                mois = "03"
            elif (re.search(r"avril", date[1], re.IGNORECASE)):
                mois = "04"
            elif (re.search(r"juin", date[1], re.IGNORECASE)):
                mois = "06"
            elif (re.search(r"juil", date[1], re.IGNORECASE)):
                mois = "07"
            elif (re.search(r"ao", date[1], re.IGNORECASE)):
                mois = "08"
            elif (re.search(r"sep", date[1], re.IGNORECASE)):
                mois = "09"
            elif (re.search(r"oct", date[1], re.IGNORECASE)):
                mois = "10"
            elif (re.search(r"nov", date[1], re.IGNORECASE)):
                mois = "11"
            elif (re.search(r"cemb", date[1], re.IGNORECASE)):
                mois = "12"
            elif (re.search(r"mai", date[1], re.IGNORECASE)):
                mois = "05"
            else:
                mois = "08"
            
            jour = date[0].strip()
            if (len(jour) == 1):
                jour = "0"+ jour
        
            date_presentation_part = annee + "-" + mois + "-" + jour
            # print("DATE PRESENTATION : " + date_presentation_part)
        
        if (debut):
            
            regex_fin = r"ANNEXE(?:S)? (?:A|À) LA D(?:E|É)LIB(?:E|É)RATION"
            if (len(re.findall(regex_fin, texte))):
                # On arrête la lecture de la délibération à la fin des articles
                # print("DETECTE FIN")
                fin = True
                break
            
            regex_paragraphe_date_seule = r"^\s*(?:\d{4}-\d{2}-\d{2}|\d{2}\/\d{2}\/\d{4})\s+\d{2}:\d{2}:\d{2}\s*$"
            if (re.search(regex_paragraphe_date_seule, texte) is not None):
                # On passe au paragraphe qui suit
                print("EXTRACTION 2 ", texte)
                continue
                
            if (y0 > limite_inf and y1 < limite_sup):
                dictionnaire_expurge['blocks'].append(block)
            
            continue
            
        else : # si pas encore début mais que pas fin
            if (not fin):
                continue
        
        # On arrive ici si debut + fin
        break
    
    dictionnaire_expurge = merge_all_blocks(dictionnaire_expurge)
    projet_delib_part = dict_to_html(dictionnaire_expurge)
    
    return (projet_delib_part, date_presentation_part, fin)

def extract_all_delib(dictionary_elements: dict, tables_camelot, verbose = False):
    
    limite_inf = 50
    limite_sup = 800
    corps_visa_delib = ""
    controle_legalite = ""
    action_budgetaire = ""
    commissions = []
    fin = False
    
    dictionnaire_expurge = {'blocks': []}
    
    last_block = 0
    
    if (tables_camelot != None):
        # DEBUT : boucle2 parcourir les tableaux trouvés dans la page courante
        for t in range(tables_camelot.n):
            print("tableau n°" + str(t))

            tableau_place = False
            table_html = ((tables_camelot[t]).df.to_html(index=False,header=False))
            table_html = table_html.replace('\xa0',' ')
            soup = BeautifulSoup(table_html,"html.parser")

            # On recupere le texte brut du tableaux t
            texte_table = soup.get_text()
            texte_table = texte_table.strip()

            # Pour obtenir le nombre de ligne dans le tableau, on a observé que les lignes sont separées par'\n\n\n'
            number_of_lines = (len(re.findall('\n\n\n', texte_table))) + 1
            texte_table = texte_table.replace('\n\n\n','\n ')
            print("number_of_lines " + str(number_of_lines))
            # DEBUT boucle3: parcourir_tous_les_paragraphes_page
            for k in range(last_block, len(dictionary_elements['blocks'])):
                # Si le bloc commence par '\n' on effectue aucun comparaison
                # print("block " + str(dictionary_elements['blocks'][k]))

                block_text = dict_block_to_text(dictionary_elements['blocks'][k]).strip()
                if (block_text == ""):
                    continue

                a = texte_table.replace("\n","").replace(" ","").find(block_text.replace("\n","").replace(" ",""))

                # Condition3_1: Verife si le tableau t doit etre place dans le paragrahe en question
                if( a>-1 and tableau_place == False):
                    # cela signifie qu'on a trouve le texte du tableau dans la ligne
                    tableau_place = True
                    numero_block = k
                    # print('block:'+str(block_list[k][5]))
                    # On incrémente le derniere block pour faciliter l'ajout d'un autre tableau au besoin
                    last_block = k
                    #print(numero_block)
                    #Permet d'avoir le nombre de colonne d'une table
                    longueur_table = len(dictionary_elements['blocks'][numero_block]['lines'])

                    # Les boucles suivantes (imbriquées) permettent de parcourir les lignes de textes contenues
                    # dans dans les paragraphes, ce qui correspond au lignes et colonnes des tables

                    # DEBUT boucle4: parcourir_tous_les paragraphe_par blocks_
                    for j in range(number_of_lines):
                        #DEBUT boucle5: Permet de parcourir chaque lignes des blocks
                        for p in range(0, longueur_table):
                            if (j == 0 and p==0):
                                # Ajoute le tableau
                                # dictionary_elements['blocks'][numero_block] = { "type": 0, "bbox": (0, 0, 0, 0), "lines": [] }
                                """
                                "lines": ({
                                    "wmode": 0,
                                    "dir": (1.0, 0.0),
                                    "bbox": (50.0, 88.17500305175781, 166.1709747314453, 103.28900146484375),
                                    "spans": ({
                                        "size": 11.0,
                                        "flags": 0,
                                        "font": "Helvetica",
                                        "color": 0,
                                        "origin": (50.0, 100.0),
                                        "text": "Some text on first page.",
                                        "bbox": (50.0, 88.17500305175781, 166.1709747314453, 103.28900146484375)
                                    })
                                })

                                """
                                dictionary_elements['blocks'][numero_block]['lines'][0]['spans'][0]['text'] = '<div class="table-camelot">' + table_html + '</div>'

                            elif((numero_block+j) < len(dictionary_elements['blocks'])):
                                dictionary_elements['blocks'][numero_block+j] = { "type": 0, "bbox": (0, 0, 0, 0), "lines": [] }
                                # dictionary_elements['blocks'][numero_block+j]['lines'][p]['spans'][0]['text'] = ""
                        # FIN boucle5
                    # FIN boucle4
                # FIN if Condition3_1
            # FIN boucle3: parcourir_tous_les_paragraphes_page_i       

        # FIN : boucle2 parcourir les tableaux trouvés dans la page courante

    for block in dictionary_elements['blocks']:

        x0, y0, x1, y1 = block['bbox']
        
        texte = ""
        for line in block['lines']:
            for span in line['spans']:
                texte += " " + span['text']
        
        regex_fin = r"ANNEXE(?:S)? (?:A|À) LA D(?:E|É)LIB(?:E|É)RATION"
        if (len(re.findall(regex_fin, texte))):
            # if ("ANNEXE A LA DELIBERATION" in texte):
            # On arrête la lecture de la délibération à la fin des articles
            fin = True
            break
        
        # Détecter actions budgétaires
        
        if ("action" in texte):
            actions_concernees = [action for action in liste_actions_budgetaires if action in texte]
            if (len(actions_concernees)):
                action_budgetaire = actions_concernees[0]
        
        # Détecter le visa de la commission principale
        # VU l'avis de la commission des transports et des mobilités
        cherche_commissions = re.findall("l'avis de la commission (.*?) ;", texte)
        # re.search(r'Part 1(.*?)Part 3', s).group(1)
        if (len(cherche_commissions)):
            #print(cherche_commissions)
            commissions.append(cherche_commissions[0])
        
        regex_paragraphe_date_seule = r"^\s*(?:\d{4}-\d{2}-\d{2}|\d{2}\/\d{2}\/\d{4})\s+\d{2}:\d{2}:\d{2}\s*$"
        if (re.search(regex_paragraphe_date_seule, texte) is not None):
            # On passe au paragraphe qui suit
            print("EXTRACTION 3 ", texte)
            continue
            
        if (y0 > limite_inf and y1 < limite_sup):
            dictionnaire_expurge['blocks'].append(block)
            
        if ("Acte rendu exécutoire le" in texte):
            # On enregistre le contrôle de légalité
            controle_legalite = texte
            fin = True
            break
            
        """
        CONTROLE SUR LES SUPPRESSIONS LIEES AUX MARGES
        if (y0 > limite_inf and y1 < limite_sup):
            print("y0 = " + str(y0) + " et y1 = " + str(y1) + " :" + texte)
        else:
            print(" ")
            print("Supprimé: " + texte)
            print(" ")
        """
    
    dictionnaire_expurge = merge_all_blocks(dictionnaire_expurge)
    corps_visa_delib = dict_to_html(dictionnaire_expurge)
    
    #print(commissions)
    if(len(commissions)):
        commission = commissions[0]
    else:
        commission = ""
    
    return (corps_visa_delib, controle_legalite, action_budgetaire, commission, fin)

def extract_date_controle(controle_legalite: str) -> str:
    date = re.search('rendu exécutoire le (.+?), depuis réception', controle_legalite)
    if (date):
        date = date.group(1).strip()
    else:
        return ""
    # Convertir en YYYY-MM-DD
    annee = date[-4:]
    
    if (re.search(r"jan", date, re.IGNORECASE)):
        mois = "01"
    elif (re.search(r"vrier", date, re.IGNORECASE)):
        mois = "02"
    elif (re.search(r"mars", date, re.IGNORECASE)):
        mois = "03"
    elif (re.search(r"avril", date, re.IGNORECASE)):
        mois = "04"
    elif (re.search(r"juin", date, re.IGNORECASE)):
        mois = "06"
    elif (re.search(r"juil", date, re.IGNORECASE)):
        mois = "07"
    elif (re.search(r"ao", date, re.IGNORECASE)):
        mois = "08"
    elif (re.search(r"sep", date, re.IGNORECASE)):
        mois = "09"
    elif (re.search(r"oct", date, re.IGNORECASE)):
        mois = "10"
    elif (re.search(r"nov", date, re.IGNORECASE)):
        mois = "11"
    elif (re.search(r"cemb", date, re.IGNORECASE)):
        mois = "12"
    elif (re.search(r"mai", date, re.IGNORECASE)):
        mois = "05"
    else:
        mois = "08"
        
    jour = date[0:2].strip()
    if (len(jour) == 1):
        jour = "0"+ jour
    
    date = annee + "-" + mois + "-" + jour
    return date

def traite_fichier(fichier, nom_fichier, date_publication, with_tables = False, verbose = True) -> dict:
    """Analyse un fichier"""
    
    pdf = fitz.open(fichier)
    
    if (date_publication == ""):
        date_publication = date_du_jour()
        
    # Initialiser les valeurs
    identifiant = ""
    numero = ""
    titre = ""
    action_budgetaire = ""
    controle_legalite = ""
    date_decision = ""
    date_presentation = ""
    type_seance = ""
    corps_expose_rapport = ""
    corps_projet_delib = ""
    thematique = ""
    sommaire_annexes = ""
    corps_visa_delib = ""
    url1 = ""
    url2 = ""
    
    if (verbose == True): print("===== DEBUT DE FICHIER ", nom_fichier," =====")
    if (verbose == True): print("Nom du fichier : ", nom_fichier)
    
    #Est-ce un rapport ou une délibération ?
    page_de_garde = pdf.load_page(0).get_text()
    # print(page_de_garde)
    deliberation = page_de_garde.find('DÉLIBÉRATION') + page_de_garde.find('DELIBERATION')
    #rapport = ('Rapport pour' in page_de_garde OR 'le conseil' in page_de_garde OR 'la commission'in page_de_garde )
    
    if (("Rapport pour" in page_de_garde)): # or ("le conseil" in page_de_garde) or ("la commission" in page_de_garde)):
        type_document = "rapport"
        if(deliberation >= 0):
            print("===========================")
            print("PROBLEME : On a détecté un rapport alors qu'il est aussi détecté comme une délibération")
            print("Nom du fichier : ", nom_fichier)
            print("TEXTE PAGE DE GARDE =======")
            print(page_de_garde)
            print("===========================")
    elif (deliberation >= 0):
        # c'est une délibération
        type_document = "deliberation"
    elif ("Envoyé en préfecture le" in page_de_garde):
        type_document = "arrete"
    else:
        # print(page_de_garde)
        type_document = "autre"
    
    if (verbose == True): print("Type doc:", type_document, " ; Indice de DELIBERATION : ", deliberation, " ; ")
    
    # Contrôles de cohérence et message
    messages = []
    if (type_document == "deliberation" and not "DEL" in nom_fichier):
        messages.append("DEL non présent dans le nom de fichier")
    if (type_document == "rapport" and not "RAP" in nom_fichier):
        messages.append("RAP non présent dans le nom de fichier")
    if (type_document == "deliberation" and "RAP" in nom_fichier):
        messages.append("Alerte RAP présent dans nom de délibération")
    if (type_document == "rapport" and "DEL" in nom_fichier):
        messages.append("Alerte DEL présent dans nom de rapport")
    if (type_document == "autre"):
        messages.append("Fichier non reconnu: " + nom_fichier)
    
    #
    # Analyse du rapport
    #
    if (type_document == "rapport"):
        
        # Récupérer identifiant, titre
        page = pdf.load_page(0)
        tp = page.get_textpage()
        dictionary_elements = tp.extractDICT()
        identifiant, titre = extract_id_titre_rapport(dictionary_elements, verbose)
        # OK print(identifiant)
        # OK print(titre)
        
        numero = numero_from_identifiant(identifiant)
        url1 = "https://www.iledefrance.fr/actes/deliberations/" + nom_fichier
        
        if (len(pdf) < 2):
            messages.append("Document d'une seule page")
        else :
            
            fin_expose = 0
            
            # Récupérer sommaire annexes
            page = pdf.load_page(1)
            tp = page.get_textpage()
            dictionary_elements = tp.extractDICT()
            sommaire_annexes = sommaire_to_html(extract_sommaire_annexes_rapport(dictionary_elements))
            # OK print(sommaire)
        
            # @TODO Adapter la fonction extract_expose_rapport() pour intégrer
            # la détection et l'insertion de blocks tableaux avec Camelot

            # Récupérer exposé des motifs
            corps_expose_rapport = ""
            for i in range(2, pdf.page_count):
                """
                @TODO Insérer algorithme insertion tableaux
                """
                page = pdf.load_page(i)
                tp = page.get_textpage()
                dictionary_elements = tp.extractDICT()
                if (with_tables == True):
                    tables_camelot = camelot.read_pdf(fichier, pages = str(i+1),flavor = 'lattice',password = '',split_text = False,layout_kwargs={'line_margin': False})
                    print("Page " + str(i+1) + " -- Tableaux détectés " + str(tables_camelot))
                else:
                    tables_camelot = None
                expose, fin = extract_expose_rapport(dictionary_elements, tables_camelot, verbose)
                corps_expose_rapport += expose
                if (fin):
                    fin_expose = i
                    break
            # OK print(corps_expose_rapport)
            
            # @TODO Adapter la fonction extract_projet_delib_rapport() pour intégrer
            # la détection et l'insertion de blocks tableaux avec Camelot
            
            # Récupérer date présentation rapport et projet delib
            date_presentation = ""
            corps_projet_delib = ""
            for i in range(fin_expose, pdf.page_count):
                """
                @TODO NE PAS Insérer algorithme insertion tableaux
                """
                page = pdf.load_page(i)
                tp = page.get_textpage()
                dictionary_elements = tp.extractDICT()
                # print ("DICT : " + str(dictionary_elements))
                projet_delib_part, date_presentation_part, fin = extract_projet_delib_rapport(dictionary_elements, verbose)
                corps_projet_delib += projet_delib_part
                if (date_presentation == ""):
                    date_presentation = date_presentation_part
                if (fin):
                    break
            # print("CORPS PROJET DELIB" + date_presentation + " /// " + corps_projet_delib)
    
    #
    # Analyse délibération
    #
    if (type_document == "deliberation"):
        
        # Récupérer identifiant, date
        page = pdf.load_page(0)
        text = page.get_text()
        tp = page.get_textpage()
        dictionary_elements = tp.extractDICT()
        identifiant, date_decision = extract_id_date_delib(text, verbose)
        # OK print(date_delib)
        # OK print(identifiant)
        
        numero = numero_from_identifiant(identifiant)
        url2 = "https://www.iledefrance.fr/actes/deliberations/" + nom_fichier
        
        # @TODO Adapter la fonction extract_all_delib() pour intégrer
        # la détection et l'insertion de blocks tableaux avec Camelot
        
        # Récupérer corps_visa_delib, controle_legalite, action budgetaire
        corps_visa_delib = ""
        controle_legalite = ""
        commission = ""
        for i in range(0, pdf.page_count):
            page = pdf.load_page(i)
            tp = page.get_textpage()
            dictionary_elements = tp.extractDICT()
            
            if (with_tables == True):
                tables_camelot = camelot.read_pdf(fichier, pages = str(i+1),flavor = 'lattice',password = '',split_text = False,layout_kwargs={'line_margin': False})
                print("Page " + str(i+1) + " -- Tableaux détectés " + str(tables_camelot))
            else:
                tables_camelot = None
            
            corps_visa_delib_part, controle_legalite_part, action_budgetaire, commission_part, fin = extract_all_delib(dictionary_elements, tables_camelot, verbose)
            corps_visa_delib += corps_visa_delib_part
            controle_legalite += controle_legalite_part
            if (commission == ""):
                commission = commission_part
            #print(commission)
            if (fin):
                break
        if (controle_legalite != ""):
            controle_legalite = extract_date_controle(controle_legalite)
        thematique = thematique_from_commission(commission)
        # print(thematique)
        # OK print(action_budgetaire)
        # OK print(controle_legalite)
        # OK print(corps_visa_delib)
    
    if ("CR" in identifiant):
        type_seance = "Conseil régional"
    elif ("CP" in identifiant):
        type_seance = "Commission permanente"
    else:
        messages.append("Type de séance non identifié")
    
    if (verbose == True): print("==== FIN FICHIER ", nom_fichier," ======")
    
    if (controle_legalite == "" and type_document == "deliberation"):
        controle_legalite = ocr_ancien_controle_legalite(pdf)
    
    if (date_decision != ""):
        date_affichage = date_decision
    elif (date_presentation != ""):
        date_affichage = date_presentation
    elif (controle_legalite != ""):
        date_affichage = controle_legalite
    else :
        date_affichage = date_publication
    
    dictionnaire = {"coll_nom": "Région Île-de-France",
                    "coll_siret": "23750007900312",
                    "identifiant": identifiant,
                    "numero": numero,
                    "titre": titre,
                    "type_acte": "DEL",
                    "type": type_document,
                    "theme": thematique,
                    "action_budgetaire": action_budgetaire,
                    "date_controle": controle_legalite,
                    "date_decision": date_decision,
                    "date_presentation" : date_presentation,
                    "date_publication": date_publication,
                    "date_affichage": date_affichage,
                    "type_seance": type_seance,
                    "corps_expose_rapport": corps_expose_rapport,
                    "sommaire_annexes": sommaire_annexes,
                    "corps_projet_delib": corps_projet_delib,
                    "corps_visa_delib": corps_visa_delib,
                    "messages": str(messages),
                    "nom_fichier": nom_fichier,
                    "url1": url1,
                    "url2": url2
                   }
    return dictionnaire





# Traitement

In [16]:
def smart_match(liste: list) -> dict:
    
    identifiant = ""
    numero = ""
    titre = ""
    type_document = ""
    thematique = ""
    action_budgetaire = ""
    controle_legalite = ""
    date_decision = ""
    type_seance = ""
    corps_expose_rapport = ""
    corps_projet_delib = ""
    sommaire_annexes = ""
    corps_visa_delib = ""
    date_publication = ""
    date_presentation = ""
    messages = ""
    nom_fichier = ""
    url1 = ""
    url2 = ""
    index = 0
    
    for item in liste:
        index += 1
        if (index > 2):
            messages += "['Trois documents pour même délibération']"
            break
        identifiant = item["identifiant"]
        numero = item["numero"]
        titre += item["titre"]
        
        # Si rapport ou délibération
        if (item["type"] == "rapport" or item["type"] == "deliberation"):
            type_document = "Délibérations et rapports du CR et de la CP"
            date_presentation = item["date_presentation"]
            corps_projet_delib = item["corps_projet_delib"]
        elif (item["type"] == "arrete"):
            type_document = "Arrêté"
        else :
            type_document = "Autre"
        
        thematique += item["theme"]
        action_budgetaire += item["action_budgetaire"]
        if (controle_legalite == ""):
            controle_legalite = item["date_controle"]
        if (date_publication == ""):
            date_publication = item["date_publication"]
        if (date_decision == ""):
            date_decision = item["date_decision"]
        if (type_seance == ""):
            type_seance += item["type_seance"]
        corps_expose_rapport += item["corps_expose_rapport"]
        sommaire_annexes += item["sommaire_annexes"]
        corps_visa_delib += item["corps_visa_delib"]
        messages += item["messages"]
        nom_fichier += item["nom_fichier"]
        url1 += item["url1"]
        url2 += item["url2"]
    
    if (thematique == ""):
        thematique = "Institution"
    
    dictionnaire = {"coll_nom": "Région Île-de-France",
                    "coll_siret": "23750007900312",
                    "identifiant": identifiant,
                    "numero": numero,
                    "titre": titre,
                    "type_acte": "DEL",
                    "type": type_document,
                    "theme": thematique,
                    "action_budgetaire": action_budgetaire,
                    "date_controle": controle_legalite,
                    "date_decision": date_decision,
                    "date_presentation": date_presentation,
                    "date_publication": date_publication,
                    "date_affichage": date_publication,
                    "type_seance": type_seance,
                    "corps_expose_rapport": corps_expose_rapport,
                    "corps_projet_delib": corps_projet_delib,
                    "sommaire_annexes": sommaire_annexes,
                    "corps_visa_delib": corps_visa_delib,
                    "messages": str(messages),
                    "nom_fichier": nom_fichier,
                    "url1": url1,
                    "url2": url2
                   }
    
    return dictionnaire

# Récupérer les délibs déjà publiées en open data

In [18]:
# URL API permettant de récupérer les délibérations déjà publiées
url_api_opendata_delibs = "https://data.iledefrance.fr/api/records/1.0/search/?dataset=actes-administratifs&q=&rows=10000&facet=type_acte&facet=type&facet=theme&facet=type_seance&refine.type_acte=DEL&apikey=70b1e8b9be88b35fd29c3706bf258b3b6c9071ffa4e32504793e395b"

In [19]:
# Requête de l'API Opendatasoft
response_API = requests.get(url_api_opendata_delibs)
data = response_API.text
parse_json = json.loads(data)

In [20]:
# Nombre de délibérations trouvées: parse_json['nhits']

# Arbre des résultats: parse_json['records']

# Pour chaque entrée des résultats:
# Timestamp de l'entrée: parse_json['records']['record_timestamp']
# Accès aux champs de l'entée: parse_json['records']['fields']
# Champs identifiant de l'entée: parse_json['records']['fields']['identifiant']
# Champ date_publication de l'entée: parse_json['records']['fields']['date_publication']

dictionnaire_delibs_opendata = {}
for record in parse_json['records']:
    dictionnaire_delibs_opendata[record['fields']['identifiant']] = {
        'identifiant': record['fields']['identifiant'],
        'timestamp': record['record_timestamp'],
        'date_publication':record['fields']['date_publication']
    }

print("Code de réponse de l'API : ", response_API.status_code)
print("Nombre de délibérations déjà publiées en open data : ", parse_json['nhits'])
# print("Dictionnaire des délibérations publiées en open data : ", dictionnaire_delibs_opendata)


Code de réponse de l'API :  200
Nombre de délibérations déjà publiées en open data :  5289


# Contrôles

In [22]:
# Pour contrôler la correspondance entre date_publication et timestamp
for identifiant in dictionnaire_delibs_opendata:
    continue
    correspondance = re.search(dictionnaire_delibs_opendata[identifiant]['date_publication'], dictionnaire_delibs_opendata[identifiant]['timestamp'])
    if (correspondance is None):
        print("Problème de correspondance : ", identifiant, dictionnaire_delibs_opendata[identifiant]['date_publication'], dictionnaire_delibs_opendata[identifiant]['timestamp'])
    else:
        print("Ok correspondance", identifiant)

# Exécution

In [24]:
type_seance = "CP"
inclure_sous_dossiers = False
date_publication = "" #"2022-09-08" #format AAAA-MM-JJ, peut être vide (dans ce cas sera remplacé par la date du jour)

os.chdir("C:/Users/ancarette/Python scripts/PUBLICATION DES ACTES/Delibs")
fileDirOrigin = "C:/Users/ancarette/Python scripts/PUBLICATION DES ACTES/Delibs"
fichier_a_inscrire = fileDirOrigin + "/extrait_delibs_" + type_seance + ".csv"

if (inclure_sous_dossiers):
    liste_fichier = []
    # TODO corriger car ne fonctionne pas dans les sous-répertoires
    for path, subdirs, files in os.walk(fileDirOrigin):
        for name in files:
            if name.endswith('.pdf'):
                #print(os.path.join(path, name))
                pikepdf_decrypte_fichier(name)
                print("Fichier traité par pike : " + str(name), ". os.path.join(path, name) = ", os.path.join(path, name))
                liste_fichier.append([os.path.join(path, name),name])
else :
    liste_fichier = []
    # [_ for _ in os.listdir(fileDirOrigin) if _.endswith('.pdf')]
    for file in os.listdir(fileDirOrigin):
        if file.endswith('.pdf'):
            pikepdf_decrypte_fichier(file)
            print("Fichier traité par pike : " + str(file))
            liste_fichier.append([os.path.join(fileDirOrigin, file), file])
    

Fichier traité par pike : CP2025-109DEL.pdf
Fichier traité par pike : CP2025-109RAP.pdf
Fichier traité par pike : CP2025-165DEL.pdf
Fichier traité par pike : CP2025-165RAP.pdf
Fichier traité par pike : CP2025-176DEL.pdf
Fichier traité par pike : CP2025-176RAP.pdf
Fichier traité par pike : CP2025-177DEL.pdf
Fichier traité par pike : CP2025-177RAP.pdf
Fichier traité par pike : CP2025-179DEL.pdf
Fichier traité par pike : CP2025-179RAP.pdf
Fichier traité par pike : CP2025-193DEL.pdf
Fichier traité par pike : CP2025-193RAP.pdf
Fichier traité par pike : CP2025-194DEL.pdf
Fichier traité par pike : CP2025-194RAP.pdf
Fichier traité par pike : CP2025-196DEL.pdf
Fichier traité par pike : CP2025-196RAP.pdf
Fichier traité par pike : CP2025-197DEL.pdf
Fichier traité par pike : CP2025-197RAP.pdf
Fichier traité par pike : CP2025-199DEL.pdf
Fichier traité par pike : CP2025-199RAP.pdf
Fichier traité par pike : CP2025-200DEL.pdf
Fichier traité par pike : CP2025-200RAP.pdf
Fichier traité par pike : CP2025

In [25]:
field_names_csv = ['coll_nom','coll_siret','identifiant','numero','titre','type_acte','type','theme','action_budgetaire',
                   'date_presentation','date_controle','date_decision','date_publication','date_affichage','type_seance',
                   'corps_expose_rapport','corps_projet_delib','sommaire_annexes','corps_visa_delib','messages','nom_fichier',
                   'url1', 'url2']

# Appel principal

In [27]:

liste_avant_identifiants = []

# Paramètre modifiable (bool) pour lancer l'algorithme Camelot ou non
with_tables_camelot = False
# Paramètre modifiable (bool) pour générer un retour verbeux
verbose = True

for fichier in liste_fichier:
    dictionnaire = traite_fichier(fichier[0], fichier[1], date_publication, with_tables_camelot, verbose)
    print("Fichier traitement fini : " + str(fichier))
    liste_avant_identifiants.append(dictionnaire)



===== DEBUT DE FICHIER  CP2025-109DEL.pdf  =====
Nom du fichier :  CP2025-109DEL.pdf
Type doc: deliberation  ; Indice de DELIBERATION :  34  ; 
EXTRACTION 3   09/07/2025 04:49:16
Dernier bloc qu'on ajoute
 //LIGNE//  Article 1 : Présentation du rapport annuel relatif à la délégation de service public de l’île de
---
Dernier bloc qu'on ajoute
 //LIGNE//  Acte rendu exécutoire le 9 juillet 2025, depuis réception en préfecture de la région Île-de-France le 9 juillet 2025  //LIGNE//  (référence technique : 075-237500079-20250709-lmc1234948-DE-1-1) et affichage ou notification le 9 juillet 2025.
---
==== FIN FICHIER  CP2025-109DEL.pdf  ======
Fichier traitement fini : ['C:/Users/ancarette/Python scripts/PUBLICATION DES ACTES/Delibs\\CP2025-109DEL.pdf', 'CP2025-109DEL.pdf']
===== DEBUT DE FICHIER  CP2025-109RAP.pdf  =====
Nom du fichier :  CP2025-109RAP.pdf
Type doc: rapport  ; Indice de DELIBERATION :  -2  ; 
EXTRACTION   EXPOSÉ DES MOTIFS
EXTRACTION   2025-06-26 16:48:05 
Dernier bloc qu'o

In [28]:

# Faire une liste d'identifiants classés
liste_identifiants = []
for dictionnaire in liste_avant_identifiants:
    if (dictionnaire['identifiant'] != ""):
        liste_identifiants.append(dictionnaire['identifiant'])
liste_identifiants = list(dict.fromkeys(liste_identifiants))

In [29]:

liste_apres_identifiants = []
for identifiant in liste_identifiants:
    # search all items matching identifiant in liste_avant_identifiants and smart_merge them
    liste = [item for item in liste_avant_identifiants if item["identifiant"] == identifiant]
    liste_apres_identifiants.append(smart_match(liste))


In [30]:
#dictionnaire_delibs_opendata
print(len(liste_avant_identifiants))
for dictionnaire in liste_avant_identifiants:
    print(dictionnaire['identifiant'])
    #print(dictionnaire['nom_fichier'])
    #print(dictionnaire['titre'])
    if(dictionnaire['identifiant'] == "CR092-15"):
        print(dictionnaire['nom_fichier'])
        if (dictionnaire['nom_fichier'] == "CP2017-630DEL.pdf"):
            dictionnaire['identifiant'] = "CP2017-630"

32
CP2025-109
CP2025-109
CP2025-165
CP2025-165
CP2025-176
CP2025-176
CP2025-177
CP2025-177
CP2025-179
CP2025-179
CP2025-193
CP2025-193
CP2025-194
CP2025-194
CP2025-196
CP2025-196
CP2025-197
CP2025-197
CP2025-199
CP2025-199
CP2025-200
CP2025-200
CP2025-203
CP2025-203
CP2025-204
CP2025-204
CP2025-206
CP2025-206
CP2025-218
CP2025-218

CP2025-238


In [31]:

# Lister les délibérations qui sont déjà publiées en open data et vont être exclues
liste_delibs_a_exclure = []
for dictionnaire in liste_apres_identifiants:
    if (dictionnaire['identifiant'] != ""):
        if (dictionnaire['identifiant'] in dictionnaire_delibs_opendata):
            print("Identifiant déjà publié en open data : ", dictionnaire['identifiant'], " ; date initiale de publication : ", dictionnaire_delibs_opendata[dictionnaire['identifiant']]['date_publication'])
            liste_delibs_a_exclure.append(dictionnaire['identifiant'])
        else:
            continue
            print("Délibération non encore publiée: ", dictionnaire['identifiant'])




In [32]:
liste_delibs_a_exclure

[]

In [33]:

# Exclure les identifiants des délibérations déjà publiées en open data
for dictionnaire in liste_apres_identifiants:
    if (dictionnaire['identifiant'] in liste_delibs_a_exclure):
        liste_apres_identifiants.remove(dictionnaire)
        print("On a bien supprimé la délibération ", dictionnaire['identifiant'])
    

In [34]:
for item in liste_apres_identifiants:
    print("Ide : ", item['identifiant'], " TRONQUE ", item['identifiant'].replace(" ",""))

Ide :  CP2025-109  TRONQUE  CP2025-109
Ide :  CP2025-165  TRONQUE  CP2025-165
Ide :  CP2025-176  TRONQUE  CP2025-176
Ide :  CP2025-177  TRONQUE  CP2025-177
Ide :  CP2025-179  TRONQUE  CP2025-179
Ide :  CP2025-193  TRONQUE  CP2025-193
Ide :  CP2025-194  TRONQUE  CP2025-194
Ide :  CP2025-196  TRONQUE  CP2025-196
Ide :  CP2025-197  TRONQUE  CP2025-197
Ide :  CP2025-199  TRONQUE  CP2025-199
Ide :  CP2025-200  TRONQUE  CP2025-200
Ide :  CP2025-203  TRONQUE  CP2025-203
Ide :  CP2025-204  TRONQUE  CP2025-204
Ide :  CP2025-206  TRONQUE  CP2025-206
Ide :  CP2025-218  TRONQUE  CP2025-218
Ide :  CP2025-238  TRONQUE  CP2025-238


# Génération du fichier CSV

In [36]:

dictionnaire = {"coll_nom": "coll_nom",
                    "coll_siret": "coll_siret",
                    "identifiant": "identifiant",
                    "numero": "numero",
                    "titre": "titre",
                    "type_acte": "type_acte",
                    "type": "type",
                    "theme": "theme",
                    "action_budgetaire": "action_budgetaire",
                    "date_presentation": "date_presentation",
                    "date_controle": "date_controle",
                    "date_decision": "date_decision",
                    "date_publication": "date_publication",
                    "date_affichage": "date_affichage",
                    "type_seance": "type_seance",
                    "corps_expose_rapport": "corps_expose_rapport",
                    "corps_projet_delib": "corps_projet_delib",
                    "sommaire_annexes": "sommaire_annexes",
                    "corps_visa_delib": "corps_visa_delib",
                    "messages": "messages",
                    "nom_fichier": "nom_fichier",
                    "url1": "url1",
                    "url2": "url2"
                   }


append_dict_as_row(fichier_a_inscrire, dictionnaire, field_names_csv)

for dictionnaire in liste_apres_identifiants:
    append_dict_as_row(fichier_a_inscrire, dictionnaire, field_names_csv)
